## Get Data

The following code will read the .h5 files that result from running OSIRIS input deck. For this the directory for the MS folder must be specified.

In [2]:
# Imports necessary for the func osiris_open_grid_data

import h5py
import numpy as np
import sys
import os
import time


In [3]:
class vysxd_data_object(object):
	def __init__(self, h5_file):
		keys = [key for key in h5_file.keys()]
		self.DIM         = len(h5_file[keys[1]].shape)
		self.DATA        = np.array(h5_file[keys[1]])
		self.DATA_UNITS  = h5_file[keys[1]].attrs['UNITS'][0].decode("utf-8")
		self.DATA_NAME   = h5_file[keys[1]].attrs['LONG_NAME'][0].decode("utf-8")
		self.TIME        = h5_file.attrs['TIME']
		self.DT          = h5_file.attrs['DT']
		self.TIME_UNITS  = h5_file.attrs['TIME UNITS'][0].decode("utf-8")


		self.AXIS1       = np.array(h5_file[keys[0]]['AXIS1'])
		self.AXIS1_UNITS = h5_file[keys[0]]['AXIS1'].attrs['UNITS'][0].decode("utf-8")
		self.AXIS1_NAME  = h5_file[keys[0]]['AXIS1'].attrs['LONG_NAME'][0].decode("utf-8")
		self.NX          = h5_file[keys[1]].shape[self.DIM-1]
		self.DX          = (self.AXIS1[1]-self.AXIS1[0])/(self.NX)
		# self.X           = np.linspace(self.AXIS1[0], self.AXIS1[-1] - self.DX, num = self.NX)
		self.X           = np.linspace(self.AXIS1[0] + self.DX/2, self.AXIS1[-1] - self.DX/2, num = self.NX)
		# self.X           = self.AXIS1[0] + np.arange(self.NX) * self.DX

		if self.DIM > 1:
			self.AXIS2       = np.array(h5_file[keys[0]]['AXIS2'])
			self.AXIS2_UNITS = h5_file[keys[0]]['AXIS2'].attrs['UNITS'][0].decode("utf-8")
			self.AXIS2_NAME  = h5_file[keys[0]]['AXIS2'].attrs['LONG_NAME'][0].decode("utf-8")
			self.NY          = h5_file[keys[1]].shape[self.DIM-1 - 1]
			self.DY          = (self.AXIS2[1]-self.AXIS2[0])/(self.NY)
			self.Y   = np.linspace(self.AXIS2[0] + self.DY/2, self.AXIS2[-1] - self.DY/2, num = self.NY)
			# self.Y   = np.linspace(self.AXIS2[0], self.AXIS2[-1] - self.DY, num = self.NY)
			# self.Y           = self.AXIS2[0] + np.arange(self.NY) * self.DY

		if self.DIM == 3:
			self.AXIS3       = np.array(h5_file[keys[0]]['AXIS3'])
			self.AXIS3_UNITS = h5_file[keys[0]]['AXIS3'].attrs['UNITS'][0].decode("utf-8")
			self.AXIS3_NAME  = h5_file[keys[0]]['AXIS3'].attrs['LONG_NAME'][0].decode("utf-8")
			self.NZ          = h5_file[keys[1]].shape[self.DIM-1 - 2]
			self.DZ          = (self.AXIS3[1]-self.AXIS3[0])/(self.NZ)
			self.Z   = np.linspace(self.AXIS3[0] + self.DZ/2, self.AXIS3[-1] - self.DZ/2, num = self.NZ)
			# self.Z   = np.linspace(self.AXIS3[0], self.AXIS3[-1] - self.DZ, num = self.NZ)
			# self.Z           = self.AXIS3[0] + np.arange(self.NZ) * self.DZ

class vysxd_raw_object(object):
	def __init__(self, h5_file):
		self.DIM         = len(h5_file.attrs['XMIN'])
		self.TIME        = h5_file.attrs['TIME'][0]
		self.TIME_UNITS  = h5_file.attrs['TIME UNITS'][0]
		self.NX          = h5_file.attrs['NX'][0]
		self.DT          = h5_file.attrs['DT'][0]
		self.DT          = h5_file.attrs['DT'][0]

		self.AXIS1       = np.array(h5_file[h5_file.keys()[0]]['AXIS1'])
		self.AXIS1_UNITS = h5_file[h5_file.keys()[0]]['AXIS1'].attrs['UNITS'][0]
		self.AXIS1_NAME  = h5_file[h5_file.keys()[0]]['AXIS1'].attrs['LONG_NAME'][0]
		self.NX          = h5_file[h5_file.keys()[1]].shape[self.DIM-1]
		self.DX          = (self.AXIS1[1]-self.AXIS1[0])/(self.NX-1)
		self.X   = np.linspace(self.AXIS1[0], self.AXIS1[1], num = self.NX)

		if self.DIM > 1:
			self.AXIS2       = np.array(h5_file[h5_file.keys()[0]]['AXIS2'])
			self.AXIS2_UNITS = h5_file[h5_file.keys()[0]]['AXIS2'].attrs['UNITS'][0]
			self.AXIS2_NAME  = h5_file[h5_file.keys()[0]]['AXIS2'].attrs['LONG_NAME'][0]
			self.NY          = h5_file[h5_file.keys()[1]].shape[self.DIM-1 - 1]
			self.DY          = (self.AXIS2[1]-self.AXIS2[0])/(self.NY-1)
			self.Y   = np.linspace(self.AXIS2[0], self.AXIS2[1], num = self.NY)

		if self.DIM == 3:
			self.AXIS3       = np.asarray(h5_file[h5_file.keys()[0]]['AXIS3'])
			self.AXIS3_UNITS = h5_file[h5_file.keys()[0]]['AXIS3'].attrs['UNITS'][0]
			self.AXIS3_NAME  = h5_file[h5_file.keys()[0]]['AXIS3'].attrs['LONG_NAME'][0]
			self.NZ          = h5_file[h5_file.keys()[1]].shape[self.DIM-1 - 2]
			self.DZ          = (self.AXIS3[1]-self.AXIS3[0])/(self.NZ-1)
			self.Z   = np.linspace(self.AXIS3[0], self.AXIS3[1], num = self.NZ)


def vysxd_get_data(filename):
	# Open first hdf5 file, save its contents to vys_data object, and close file
	file = h5py.File(filename, 'r')
	vysxd_data = vysxd_data_object(file)
	file.close()

	return vysxd_data


def get_osiris_quantity_1d(quant_path, n_start = 0, n_end = -1, n_skip = 1):
    quant_list = np.sort(np.array(os.listdir(quant_path)))
    quant_list = quant_list[np.logical_not(quant_list == '.DS_Store')]
    quant_list = quant_list[n_start:n_end:n_skip]
    
    n_t = len(quant_list)
    quant = vysxd_get_data(quant_path + quant_list[0])
    n_x = quant.NX
    dx = quant.X[1]-quant.X[0]
    
    Q = np.zeros((n_t, n_x), dtype = np.float32)
    t = np.zeros(n_t)
    Q[0,:] = quant.DATA
    t[0]   = quant.TIME[0]
    for i in range(1, n_t):
        quant = vysxd_get_data(quant_path + quant_list[i])
        Q[i,:] = quant.DATA
        t[i] = quant.TIME[0]
    
    dt = t[1]-t[0]
    
    X = quant.X
    
    return [Q, dt, dx, t, X]

In [4]:
def get_fld_data(base_dir, n_start = 0, n_end = -1, n_skip = 1):
    
    fld_data = {} # data dictionary
    print("Getting fields... ")
    fld_data['E1'], dt, dx, t_arr, x_arr = get_osiris_quantity_1d(base_dir + 'FLD/part_e1/', n_start, n_end, n_skip)
    fld_data['E2'], dt, dx, t_arr, x_arr = get_osiris_quantity_1d(base_dir + 'FLD/part_e2/', n_start, n_end, n_skip)
    fld_data['E3'], dt, dx, t_arr, x_arr = get_osiris_quantity_1d(base_dir + 'FLD/part_e3/', n_start, n_end, n_skip)
    #fld_data['B1'], dt, dx, t_arr, x_arr = get_osiris_quantity_1d(base_dir + 'FLD/part_b1/', n_start, n_end, n_skip)
    fld_data['B2'], dt, dx, t_arr, x_arr = get_osiris_quantity_1d(base_dir + 'FLD/part_b2/', n_start, n_end, n_skip)
    fld_data['B3'], dt, dx, t_arr, x_arr = get_osiris_quantity_1d(base_dir + 'FLD/part_b3/', n_start, n_end, n_skip)

    print('time step = ', dt)
    print('spatial resolution = ', dx)
    
    fld_data['x'] = x_arr
    fld_data['dx'] = dx
    fld_data['t'] = t_arr
    fld_data['dt'] = dt
    
    return fld_data

In [ ]:
fld_data = 